In [1]:
import os
base_path = os.path.dirname(os.path.abspath(""))
base_path = base_path + "/headers/"
print(base_path)
header_path = base_path + 'header_data_treatment.ipynb'
%run $header_path

C:\Users\common\projet_3_lru\sorting_univ\script/headers/
initalizing spark session 


In [2]:
df_inst = (spark.read.format('parquet').load('file:\\' + main_path_openalex + 'institutions.parquet')
          )

In [3]:
inst = spark.read.parquet('file:\\' + save_path + 'inst_fr.parquet')

In [82]:
fr_esr = spark.read.format('csv').option("delimiter", ";").option("header","true").load('file:\\' + scanR_path + '\\fr-esr-principaux-etablissements-enseignement-superieur.csv')

In [83]:
fr_esr = (fr_esr
          .select(func.col('libellé').alias('name'), func.col('nom court').alias('short_name'),
                  func.col('identifiant_idref').alias('idref'), func.col('Identifiant ROR').alias('ror'),
                  func.col('Identifiant wikidata').alias('wikidata'), func.col("type d'établissement").alias('type'),
                 func.col("secteur d'établissement").alias('secteur'))
          .distinct()
         )
fr_esr.show()
          

+--------------------+--------------------+---------+---------+---------+-------------------+-------+
|                name|          short_name|    idref|      ror| wikidata|               type|secteur|
+--------------------+--------------------+---------+---------+---------+-------------------+-------+
|Université de Rennes|          UNIVRENNES|26693823X|015m7wh34| Q1987282|         Université| public|
|Institut français...|                IFAO|026387395|055pd2p26| Q1472454|Autre établissement| public|
|Université Paris ...|             UP Cité|236453505|05f82e368|Q55849612|         Université| public|
|École des hautes ...|           HEC Paris|027321118|0423jsj19|  Q273535|              École|  privé|
|Institut national...|        INSA de Lyon|052444724|050jn9y42| Q1633859|              École| public|
|Université de Poi...| Université Poitiers|026403765|04xhy8q59|  Q661056|         Université| public|
|Neoma Business Sc...|            Neoma BS|250789329|01g1pe685|Q16665962|         

In [111]:
merged_by_ror = (df_inst
                 .select('id', 'ror')
                 .join(fr_esr
                       .withColumn('ror', func.concat(func.lit('https://ror.org/'), func.col('ror'))), on = 'ror', how ='inner')
                 .select(func.regexp_replace(func.col('id'), 'https://openalex.org/', '').alias('inst_id'),'idref','name','ror','type',"secteur")
                 .distinct()
                )
merged_by_ror.count()

215

In [112]:
merged_by_ror.show()

+-----------+-------------------+--------------------+--------------------+-------------------+-------+
|    inst_id|              idref|                name|                 ror|               type|secteur|
+-----------+-------------------+--------------------+--------------------+-------------------+-------+
| I187986737|          026370921|   Collège de France|https://ror.org/0...|Grand établissement| public|
| I138816502|          029473284|Université de Cor...|https://ror.org/0...|         Université| public|
|I2802487185|          030969379|Université du Lit...|https://ror.org/0...|         Université| public|
|I2746051580|          241597595|Université Paris ...|https://ror.org/0...|Grand établissement| public|
| I110017253|          026404478| Université de Tours|https://ror.org/0...|         Université| public|
|I4210103002|          252404955|Université Clermo...|https://ror.org/0...|         Université| public|
|I4210145102|          238327159|Institut polytech...|https://ro

In [80]:
merged_by_ror.groupBy('type').count().show()

+-------------------+-----+
|               type|count|
+-------------------+-----+
|         Université|   61|
|Grand établissement|   27|
|              École|  116|
|Autre établissement|   10|
+-------------------+-----+



In [87]:
df_inst.filter(func.col('id').contains("I4403386658")).show()

+-----------------------+--------------+------------+----------+--------------+------------+------------+---------------------+-------------------------+---+------------+---+---+-------------------+---------+-------------+---------------+-------+------------+-----+---+-------------+-----------+------+----+-------+-------+------------+-------------+-----------+----------+
|associated_institutions|cited_by_count|country_code|country_id|counts_by_year|created_date|display_name|display_name_acronyms|display_name_alternatives|geo|homepage_url| id|ids|image_thumbnail_url|image_url|international|is_super_system|lineage|repositories|roles|ror|summary_stats|topic_share|topics|type|type_id|updated|updated_date|works_api_url|works_count|x_concepts|
+-----------------------+--------------+------------+----------+--------------+------------+------------+---------------------+-------------------------+---+------------+---+---+-------------------+---------+-------------+---------------+-------+------

In [73]:
merge = (inst.withColumn('to_join',
                                           func.regexp_replace(func.lower(func.col('name')),       " de| la| d'| l'| des| du| le| les",'')
                )
      .withColumn('to_join', func.regexp_replace(func.col('to_join'), 'é|è', 'e'))
.join(fr_esr.filter(  (~ func.col('ror').isin(merged_ror)) | (func.col('ror').isNull()) )
      .withColumn('to_join', 
                  func.explode(func.array([func.regexp_replace(func.lower(func.col('name')),       " de| la| d'| l'| des| du| le| les",''), 
                                           func.regexp_replace(func.lower(func.col('short_name')), " de| la| d'| l'| des| du| le| les",'')])))
      .withColumn('to_join', func.regexp_replace(func.col('to_join'), 'é|è', 'e'))
      , on = 'to_join', how ='inner'))
merge.count()

3

In [74]:
merge.show()

+--------------------+-----------+--------------------+---------+-------------+--------------------+---------------+--------------------+-----+-------+----+--------------------+----------+---------+----+--------+----------+
|             to_join|    inst_id|                name|     type|         city|         topic_share|     main_topic|              parent|fused|uni_pub|cnrs|                name|short_name|    idref| ror|wikidata|      type|
+--------------------+-----------+--------------------+---------+-------------+--------------------+---------------+--------------------+-----+-------+----+--------------------+----------+---------+----+--------+----------+
| universite toulouse|  I17866349|Université de Tou...|education|     Toulouse|[{Chemistry, 0.00...|    Engineering|[{I17866349, Univ...|    0|      1|   0|Université de Tou...|      NULL|     NULL|NULL|    NULL|Université|
|institut protesta...|I4210121454|Institut Protesta...|education|  Montpellier|[{Social Sciences...|Soci

In [76]:
inst.filter(func.col('inst_id')=="I4403386658").show()

+-------+----+----+----+-----------+----------+------+-----+-------+----+
|inst_id|name|type|city|topic_share|main_topic|parent|fused|uni_pub|cnrs|
+-------+----+----+----+-----------+----------+------+-----+-------+----+
+-------+----+----+----+-----------+----------+------+-----+-------+----+



In [70]:
merged_idref = [row[0] for row in merged_by_ror.select('idref').collect() if row[0] !=None]
merged_ror = [row[0].replace('https://ror.org/','') for row in merged_by_ror.select('ror').collect() if row[0] !=None]

In [54]:
merged_idref

['203592077',
 '242335675',
 '027771881',
 '050425900',
 '026523493',
 '060121076',
 '032760639',
 '026394944',
 '05017746X',
 '131056549',
 '027321118',
 '221333754',
 '026403714',
 '026374986',
 '03082057X',
 '029473284',
 '034641491',
 '163078998',
 '259265152',
 '136233902',
 '028064089',
 '030820529',
 '157779092',
 '156395703',
 '241345251',
 '09560846X',
 '240648315',
 '052444724',
 '114146756',
 '029823382',
 '026403765',
 '031308570',
 '026404478',
 '035375043',
 '026523248',
 '250789329',
 '027361837',
 '033364346',
 '034634894',
 '034635491',
 '027404978',
 '252404955',
 '034817670',
 '026370921',
 '175330263',
 '026403552',
 '035348127',
 '027941426',
 '026403919',
 '157040569',
 '026404435',
 '187841578',
 '031122337',
 '026403021',
 '076359735',
 '26693823X',
 '027976122',
 '030646480',
 '236453505',
 '026388766',
 '026402920',
 '026404451',
 '026537036',
 '25843614X',
 '026402971',
 '149154992',
 '026386291',
 '199743169',
 '030507200',
 '026403250',
 '034656278',
 '0264

In [60]:
fr_esr.count()

246

In [88]:
fr_esr.filter(  (~ func.col('ror').isin(merged_ror)) | (func.col('ror').isNull()) ).show(100,truncate = False)

+-------------------------------------------------------------------------------------------+-------------------------+---------+---------+----------+----------+-------+
|name                                                                                       |short_name               |idref    |ror      |wikidata  |type      |secteur|
+-------------------------------------------------------------------------------------------+-------------------------+---------+---------+----------+----------+-------+
|IAE Paris - Sorbonne Business School                                                       |IAE de Paris             |067108490|NULL     |Q3151899  |École     |public |
|Université de Mayotte                                                                      |NULL                     |262921715|02t0k0832|Q2946102  |Université|public |
|École d'ingénieurs des sciences aérospatiales                                              |ELISA                    |NULL     |NULL     |Q65162999 |

In [89]:
sorted_by_hand = { #'IAE Paris - Sorbonne Business School':
                   'Université de Mayotte':"I4403928291",
               "École d'ingénieurs des sciences aérospatiales" :"I58581550",
    'Université de technologie Tarbes Occitanie Pyrénées':"I119865316",
    #"Institut régional du travail social de Champagne-Ardennes"
    "Université Bourgogne Europe":"I177064439",
    "Université de Montpellier Paul-Valéry":'I208215962',
    #Institut national du service public
    "Nîmes université":"I66456633",
    #Y SCHOOLS
    "École nationale supérieure de paysage":"I4387154791",
    #Brest Business School
    #École supérieure de fonderie et de forge
    #Institut national des sciences et techniques nucléaires
    #Université Marie et Louis Pasteur
    "Université de Toulouse": "I17866349",
    #Institut supérieur du commerce de Paris,
    #École spéciale d'architecture
    #Institut supérieur de l'électronique et du numérique de Brest
    #Centre national des arts du cirque
    #Institut des relations publiques et de la communication d’Angers
    #ESIEE-IT
    #Institut français de la mode 
    #École nationale supérieure des arts et techniques du théâtre
    "Institut supérieur du bâtiment et des travaux publics": 'I4210160645',
    #Centre des hautes études de la construction
    #École supérieure d'art et de design d'Amiens
    "Université Jean Monnet":"I86767153",
    #Centre de ressources, d'expertise et de performance sportives de Toulouse 
    "École nationale supérieure du pétrole et des moteurs":"I4403386658",
    "Institut protestant de théologie":"I4210121454",
    "Institut supérieur international du parfum, de la cosmétique et de l'aromatique alimentaire":"I4387154221",
    "ICN Business School": "I4400600934",
    "École nationale supérieure des métiers de l'image et du son":'I4387156185',
    #Institut d'ingénierie informatique de Limoges
    "École de management Léonard De Vinci":"I4210121459"
    
                }

In [94]:
print([test[1]for test in sorted_by_hand.items()])

['I4403928291', 'I58581550', 'I119865316', 'I177064439', 'I208215962', 'I66456633', 'I4387154791', 'I17866349', 'I4210160645', 'I86767153', 'I4403386658', 'I4210121454', 'I4387154221', 'I4400600934', 'I4387156185', 'I4210121459']


In [101]:
expr_merge_hand = 'CASE WHEN ' + ' WHEN '.join(['name = "' + el[0] + '" THEN "' + el[1] + '"' for el in sorted_by_hand.items()] )+ ' END'
expr_merge_hand                  

'CASE WHEN name = "Université de Mayotte" THEN "I4403928291" WHEN name = "École d\'ingénieurs des sciences aérospatiales" THEN "I58581550" WHEN name = "Université de technologie Tarbes Occitanie Pyrénées" THEN "I119865316" WHEN name = "Université Bourgogne Europe" THEN "I177064439" WHEN name = "Université de Montpellier Paul-Valéry" THEN "I208215962" WHEN name = "Nîmes université" THEN "I66456633" WHEN name = "École nationale supérieure de paysage" THEN "I4387154791" WHEN name = "Université de Toulouse" THEN "I17866349" WHEN name = "Institut supérieur du bâtiment et des travaux publics" THEN "I4210160645" WHEN name = "Université Jean Monnet" THEN "I86767153" WHEN name = "École nationale supérieure du pétrole et des moteurs" THEN "I4403386658" WHEN name = "Institut protestant de théologie" THEN "I4210121454" WHEN name = "Institut supérieur international du parfum, de la cosmétique et de l\'aromatique alimentaire" THEN "I4387154221" WHEN name = "ICN Business School" THEN "I4400600934" WH

In [103]:
merged_by_hand = (fr_esr
                  .filter(  (~ func.col('ror').isin(merged_ror)) | (func.col('ror').isNull()) )
                  .withColumn('inst_id', func.expr(expr_merge_hand))
                 )
merged_by_hand.show()

+--------------------+--------------------+---------+---------+----------+----------+-------+-----------+
|                name|          short_name|    idref|      ror|  wikidata|      type|secteur|    inst_id|
+--------------------+--------------------+---------+---------+----------+----------+-------+-----------+
|IAE Paris - Sorbo...|        IAE de Paris|067108490|     NULL|  Q3151899|     École| public|       NULL|
|Université de May...|                NULL|262921715|02t0k0832|  Q2946102|Université| public|I4403928291|
|École d'ingénieur...|               ELISA|     NULL|     NULL| Q65162999|     École|  privé|  I58581550|
|Université de tec...|                NULL|     NULL|     NULL|Q124412825|     École| public| I119865316|
|Institut régional...|IRTS Champagne-Ar...|158736869|     NULL|      NULL|     École|  privé|       NULL|
|Université Bourgo...|                NULL|     NULL|     NULL|      NULL|Université| public| I177064439|
|Université de Mon...|                NULL|   

In [113]:
merged = (merged_by_ror
          .select('inst_id', 'idref', 'name', 'type','secteur')
          .unionAll(merged_by_hand.select('inst_id', 'idref','name', 'type','secteur'))
          .distinct()
         )
merged.show()

+-----------+-------------------+--------------------+-------------------+-------+
|    inst_id|              idref|                name|               type|secteur|
+-----------+-------------------+--------------------+-------------------+-------+
|  I51101395|          027361802|Université Paris ...|         Université| public|
|  I90183372|          157040569|Université de Lor...|Grand établissement| public|
| I277688954|          241345251|Université Paris-...|         Université| public|
|I4210126056|086908480,193857561|Institut national...|Autre établissement| public|
|  I68947357|          131056549|Université de Str...|         Université| public|
|I4210103002|          252404955|Université Clermo...|         Université| public|
|  I24715933|          026404451|Université de La ...|         Université| public|
| I161463649|          028222814|Audencia Business...|              École|  privé|
|I4210123735|          028888537|    Sciences Po Lyon|Autre établissement| public|
|I43

In [114]:
merged.count()

251

In [2]:
idex = {"idex_aix_marseille": #https://anr.fr/ProjetIA-11-IDEX-0001
        ['I21491767',"I4210142724",'I4210087487','I4387155629', # Aix-Marseille Université, Ecole centrale de Marseille, CHRU Marseille, IEP Aix 
                             # "I4210166444", "I4387153840","I4210128565"  #IRD Marseille, Inserm Paris, CEA Saclay
        ]
,
        'idex_paris_saclay':#https://anr.fr/ProjetIA-11-IDEX-0003
        ['I195731000',"I22248866", "I83860666", "I201181511","I11559806","I142476485","I4210091746","I205703379", # UVSQ, AgroParisTech, HEC,ENSTA, ENS Saclay, X, SupOptique, Télécom,
         "I84009706", "I4210088668", "I2801658355" ,"I2801148539","I4210107720","I88467170", "I4210126360"#ENSAE,INRAE, ONERA, Synchotron Soleil, Centrale Supélec, Univ Evry Val d'Essonne, Inria Saclay
         #,"I4387153840", "I4210128565"# Inserm paris,CEA Saclay
        ] 
         # missing : IHES,Systematic,Institut Villebon Georges Charpak, ADEPRINA
,      'idex_bordeaux' : #https://anr.fr/ProjetIA-10-IDEX-0003
        ['I3130945570',"I4210160189",'I139506611','I4210091158' #Univ Bordeaux Montaigne, Polytechnique Bordeaux, Sciences Po Bordeaux, ENSA Bordeaux
         #,"I4387153840", "I4210128565"# Inserm paris,CEA Saclay
        ]
,      "idex_psl": #https://anr.fr/ProjetIA-10-IDEX-0001
        ["I2746051580", #PSL
            'I187986737',"I149758196","I29607241",'I98910050',"I80043",'I2800004676' #CdF, Chimie ParisTech, ENS Ulm, ESPCI, Institut Curie, Observatoire de Paris
         , 'I56435720',"I4210166784","I4210107334","I4210118986","I4210126561" #Dauphine, Ecole Nat Sup Arts Deco, Beaux Arts Paris, conservatoire musique et danse, institut louis bachelier
         ,"I70768539", "I159885104","I4210127304","I4387156185", "I90669466","I4210092825","I4210100316" #Mines Paris, EPHE, fondation Rotschild, fémis, EHESS, Ecole Française d'Extrême Orient, Chartes, 
      #  , "I4210126360"#Inria Saclay
        ]
,      "idex_paris_cite": #https://anr.fr/ProjetIA-18-IDEX-0001
        ['I204730241',"I148261361","I4210091279","I205092303","I162346809","I1325388108",'I182627622',"I1326498283" #Paris Cité, institut de physique du globe, Sorbonne Paris-Nord, Sciences Po, INALCO, INED, Sorbonne Nouvelle, INRIA
         #,"I4210166444","I4387153840","I4210128565" # IRD Marseille, Inserm Paris, CEA Saclay
        ]
,      "idex_sorbonne_univ": #https://anr.fr/ProjetIA-11-IDEX-0004
        ["I39804081" ,"I4210123595","I102272798","I102516824", #Sorbonne Université, Museum d'histoire naturelle, UTC
         #,"I4210166444","I4387153840","I4210128565" # IRD Marseille, Inserm Paris, CEA Saclay
        ]
,      "idex_strasbourg": #https://anr.fr/ProjetIA-10-IDEX-0002
        ["I68947357" #Univ_strasbourg
                  #,"I4387153840" # Inserm paris
        ]
,      "idex_2_cote_azur": #https://anr.fr/ProjetIA-15-IDEX-0001
        [ 'I201841394','I4210147253', "I4210126779","I3018988418", "I4210111578", #UCA, CNRS côte d'azur, observatoire côte d'azur, CHU Nice, INSERM côte d'azur
         "I1902872","I90326151","I181418319" # Eurecom, EDHEC, SKEMA, 
         #,"I70768539" Mines Paris                            
         # "I4210166444"  #IRD Marseille # INRAE
        ]
,     'idex_2_grenoble': #https://anr.fr/ProjetIA-15-IDEX-0002
        ['I106785703',"I4210113225","I3020098449", "I4210150872", #Polytechnique Grenoble, ENSAG, CEA Grenoble, INSERM Alpes
         "I899635006","I2800555055","I4210101348","I4387155711","I4387154183" #UGA, CHU Grenoble, INRIA Grenoble, Sciences Po Grenoble, IRSTEA

         # "I4210155509" CNRS Alpes
        ]

,     'idex_annulee_toulouse': #https://anr.fr/ProjetIA-11-IDEX-0002
        ["I4210126056",'I17866349','I4210152422','I3131550300', #Centre Champollion, Univ Toulouse, Toulouse Jaurès,Toulouse Capitole,
         'I134560555',"I205747304","I196454796","I193033237" ,"I4210147367" # Toulouse Sabatier, Polytechnique Toulouse, INSA Toulouse, ISEA-SUPAéro, INRAE Occitanie
         , "I4210106734", "I4210145734","I4210092408","I176063091" # CNRS Occitanie, INSERM Occitanie, TSE, Ecole Vétérinaire Toulouse,
        ,"I4210091876","I119865316","I4210112425","I129162050" #Mines d'Albi, Université de Technologie de Tarbes, Sciences Po Toulouse,Toulouse Business School
         ,"I100296615",  "I100296615" #ENAC, Ecole nationale d'Ingénieurs de Purpan
         #,"I4210166444" #IRD Marseille
        ]
,     "idex_annulee_lyon":#https://anr.fr/ProjetIA-16-IDEX-0005
        ["I112936343","I203339264", "I4210166914","I154490720","I113428412" #Centrale Lyon, Univ Lyon, ENTPE, ENISE, ENS Lyon
         , "I48430043" ,"I4210123735",'I100532134', "I188626449", "I874609350","I4210104684" #INSA Lyon, Sciences Po Lyon, Lyon I, Lyon II, Lyon III, VetAgroSup
          #"I4387153840" "I4210155509" # Inserm Paris CNRS Rhône Alpes
        ]

#,     'isite_lorraine':#https://anr.fr/ProjetIA-15-IDEX-0004
#        ["I90183372","I4210100260", "I4210141980" ,"I1288283403", #univ Lorraine, chu nancy, inserm nancy,georgia tech nancy,
#         "I4210098696" #Institut Régional de médecine physique et de Réadaptation de Nancy
#        ]
#,     'isite_
,     "multi_idex": 
        ["I1294671590", "I4210166444","I4387153840","I4210128565",'I1326498283',"I70768539",'I4210155509', # CNRS, IRD Marseille, Inserm Paris, CEA Saclay, INRIA, Mines Paris, CNRS Rhône-Alpes
        ]
}


dict_items([('idex_aix_marseille', ['I21491767', 'I4210142724', 'I4210087487', 'I4387155629']), ('idex_paris_saclay', ['I195731000', 'I22248866', 'I83860666', 'I201181511', 'I11559806', 'I142476485', 'I4210091746', 'I205703379', 'I84009706', 'I4210088668', 'I2801658355', 'I2801148539', 'I4210107720', 'I88467170', 'I4210126360']), ('idex_bordeaux', ['I3130945570', 'I4210160189', 'I139506611', 'I4210091158']), ('idex_psl', ['I2746051580', 'I187986737', 'I149758196', 'I29607241', 'I98910050', 'I80043', 'I2800004676', 'I56435720', 'I4210166784', 'I4210107334', 'I4210118986', 'I4210126561', 'I70768539', 'I159885104', 'I4210127304', 'I4387156185', 'I90669466', 'I4210092825', 'I4210100316']), ('idex_paris_cite', ['I204730241', 'I148261361', 'I4210091279', 'I205092303', 'I162346809', 'I1325388108', 'I182627622', 'I1326498283']), ('idex_sorbonne_univ', ['I39804081', 'I4210123595', 'I102272798', 'I102516824']), ('idex_strasbourg', ['I68947357']), ('idex_2_cote_azur', ['I201841394', 'I4210147253'

In [8]:
expr_idex = "CASE WHEN " + ' WHEN '.join([ 'inst_id = "' + ('" OR inst_id = "').join(item[1])   + '" THEN idex = "' + item[0] +'"' for item in idex.items()]) + ' ELSE idex = "no_idex"'
expr_idex

'CASE WHEN inst_id = "I21491767" OR inst_id = "I4210142724" OR inst_id = "I4210087487" OR inst_id = "I4387155629" THEN idex = "idex_aix_marseille" WHEN inst_id = "I195731000" OR inst_id = "I22248866" OR inst_id = "I83860666" OR inst_id = "I201181511" OR inst_id = "I11559806" OR inst_id = "I142476485" OR inst_id = "I4210091746" OR inst_id = "I205703379" OR inst_id = "I84009706" OR inst_id = "I4210088668" OR inst_id = "I2801658355" OR inst_id = "I2801148539" OR inst_id = "I4210107720" OR inst_id = "I88467170" OR inst_id = "I4210126360" THEN idex = "idex_paris_saclay" WHEN inst_id = "I3130945570" OR inst_id = "I4210160189" OR inst_id = "I139506611" OR inst_id = "I4210091158" THEN idex = "idex_bordeaux" WHEN inst_id = "I2746051580" OR inst_id = "I187986737" OR inst_id = "I149758196" OR inst_id = "I29607241" OR inst_id = "I98910050" OR inst_id = "I80043" OR inst_id = "I2800004676" OR inst_id = "I56435720" OR inst_id = "I4210166784" OR inst_id = "I4210107334" OR inst_id = "I4210118986" OR in